## 5 - Convolutional Sequence to Sequence Learning

Trong notebook lần này, chúng ta sẽ implementing model trong paper [Convolutional Sequence to Sequence Learning](https://arxiv.org/pdf/1705.03122.pdf).

![figure1](./images/5.enc_dec.PNG)

### Introduction

> Model trong notebook lần này khác biệt khá nhiều so với models ở các notebook trước. Lần này, chúng ta sẽ không sử dụng RNN để xây dựng model. Thay vào đó, chúng ta sử dụng Convolutional layers, thường được sử dụng trong xử lý ảnh. Tham khảo [link này](https://github.com/longlnOff/sentiment-analysis/blob/main/4.%20Convolutional%20Sentiment%20Analysis.ipynb) để có cái nhìn tổng quan về CNN cũng như cách áp dụng CNN trong bài toán sentiment analysis. Bên cạnh đó, nhấn vào [đây](https://github.com/longlnOff/sentiment-analysis) để tìm hiểu thêm về bài toán sentiment analysis và các phương pháp xử lý.

> Về cơ bản, convolutional layers sử dụng các filters. Các filters này có chiều rộng (width) và chiều cao (height). Nếu filter có width bằng 3, nó có thể quan sát 3 tokens liên tiếp. Mỗi convolutional layer có rất nhiều filter (trong notebook này là 1024 filters). Mỗi filter trượt dọc theo sequence, từ đầu tới cuối và quan sát 3 tokens đúng liền kề nhau ở mỗi lần trượt (trong trường hợp filter có width bằng 3). **Ý tưởng chính ở đây là: Mỗi một filter trong 1024 filters sẽ trích xuất ra đặc tính khác nhau từ văn bản. Kết quả của quá trình trích xuất đặc trưng này sẽ model sử dụng làm đầu vào cho các convolutional layers khác. Nhiều lớp convolutional sẽ trích xuất nhiều đặc trưng từ source sentence để dịch nó sang target sequence.

### Preparing the Data

> Import các module cần thiết và thiết lập random seeds.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Load spacy models và định nghĩa tokenizers cho source sentence và target sentence.

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

> Tiếp theo, ta thiết lập `Field`s để xác định cách dữ liệu được xử lý. Mặc định, các RNN models trong Pytorch yêu cầu input sequence là tensor có kích thước **[sequence length, batch size]**, và torchtext cũng mặc định trả về batches các tensor có kích thước tương tự. Tuy nhiên, trong notebook này, chúng ta sử dụng CNNs, và CNNs yêu cầu input là tensor có kích thước **[batch size, sequence length]**. Ta thông báo cho torchtext chuyển tensor mặc định sang tensor có kích thước  **[batch size, sequence length]** bằng cách thiết lập tham số `batch_first = True`.

> Bên cạnh đó, chúng ta cũng sẽ thực hiện chuyển các tokens về lower text và chèn token bắt đầu và token kết thúc sequence.

In [ ]:
SRC = Field(tokenize = tokenize_de,
            init_token='',
            eos_token='',
            lower=True,
            batch_first=True)

TRG = Field(tokenize = tokenize_en,
            init_token='',
            eos_token='',
            lower=True,
            batch_first=True)

> Tiếp theo, ta thực hiện load dataset.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

> Build Vocabulary, coi các tokens xuất hiện ít hơn 2 lần trong corpus là \<unk> token.

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

> Bước cuối cùng trong công đoạn chuẩn bị dữ liệu là xây dựng iterator.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
train_iterator, valid_iterator, test_iterator = \
        BucketIterator.splits(
            (train_data, valid_data, test_data),
            batch_size=BATCH_SIZE,
            device=device)


### Building the Model

> Tiếp theo, ta thực hiện xây dựng model. Tương tự các notebook trước, model được tạo bởi 2 khối: encoder và decoder. Khối encoder thực hiện encodes input sentence, ở dạng source language, thành *context vector*. Khối decoder sẽ decodes context vector để sinh ra output sentence ở dạng target language.

##### Encoder

> Models ở các notebook trước sử dụng encoder để nén toàn bộ thông tin của input sentence vào một single context vector,z. Tuy nhiên, convolutional seq2seq model có một chút khác biệt - với mỗi token trong input sequence, ta sẽ thu được 2 context vectors. Nếu input sentence có 6 tokens, ta sẽ thu được 12 context vectors (2 vector cho mỗi token). 

> Hai context vector thu được từ mỗi token là *conved* vector và *combined* vector. *Conved* vector là kết quả của mỗi token khi được truyền qua một vài layers. *Combined* vector là tổng của convolved vector và embedding của token đó. Cả 2  vector này được trả về từ encoder để phục vụ cho quá trình decode.

> Hình dưới minh họa việc một input sentence (zwei menschen fetchen) được truyền vào encoder.

![figure2](./images/5.enc_example.PNG)

> Trước tiên, được truyền qua *token embedding layer*. Tuy nhiên, model này không chứa các RNN nên nó không thể nhận biệt được thứ tự của các tokens bên trong sequence. Để khắc phục vấn đề này, ta có một embedding layer thứ 2 - *positional embedding layer*. Đây là một embedding layer chuẩn với đầu vào không phải là tokens, thay vào đó, ta truyền vào vị trí của tokens bên trong sequence - bắt đầu với \<sos> token, ở vị trí 0.

> Tiếp theo, token embeddings và positional embeddings được *elementwise summed* để sinh ra vector chứa thông tin về token cũng như vị trí của token đó bên trong sequence - từ nay ta sẽ gọi vector tổng hợp này là *embedding vector*. Sau đó, ta đưa *embedding vector* này qua một linear layer để biến đối nó thành vector khác có số chiều tương ứng với hidden dimension size mong muốn (đầu ra sẽ là hidden vector).

> Bước tiếp theo, ta cần truyền *hidden vector* này vào **N** *convolutionalblocks*. Chi tiết về hoạt động của *convolutional blocks* sẽ được trình bày kỹ hơn ở phần sau. Sau khi đi qua convolutional blocks, vector sẽ được đưa vào một linear layer khác để chuyển nó từ vector có kích thước bằng hidden dimension size sang embedding dimension size. Quá trình trên tạo ra ***conved*** vector - và mỗi token, ta thu được một ***conved*** vector. 

> Cuối cùng, ***conved*** vector được *elementwise summed* với embedding vector thông qua residual connection để sinh ra ***combined*** vector cho mỗi token. Và mỗi token sẽ có một ***combined*** vector.

> Như vậy, ta thu được 2 vector là: ***conved*** vector và ***combined*** vector đối với mỗi token trong input sequence.

##### Convolutional Blocks

> Phần này sẽ trình bày ngắn gọn hoạt động của convolutional blocks. 

> Hình dưới mô tả hoạt động của 2 convolutional blocks với một filter (blue) trượt ngang qua các tokens bên trong sequence. Trong notebook này, chúng ta sẽ xây dựng encoder với 10 convolutional blocks, mỗi block có 1024 filter.

![figure3](./images/5.convolutional_blocks.PNG)

> Đầu tiên, input sentence được đệm, lý do là convolutional layers sẽ giảm chiều dài của input sentence. Tuy nhiên, ở đây chúng ta mong muốn chiều dài câu đầu vào của convolutional blocks phải bằng với chiều dài output sentence của convolutional blocks. Nếu như không padding, chiều dài output sentence của convolutional blocks sẽ là: "**chiều dài câu đầu vào - `filter_size - 1`**", ngắn hơn sequence được đưa vào convolutional layer `filter_size -1` đơn vị. Ví dụ, filter có size bằng 3 thì output sentence sẽ ngắn hơn input sentence 2 đơn vị. Để khắc phục vấn đề này, ta thực hiện padding sequence ở đầu và cuối, mỗi phía 1 token. Ta có công thức tính toán số token cần đệm ở mỗi phía trong trường hợp filter size là số lẻ như sau: <br>
>> `number padding token each side = (filter_size - 1) / 2` <br>

> (Ở đây, ta không xét trường hợp filter có size là số chẵn).

> Các filter này được thiêt kế sao cho output hidden dimension gấp đôi input hidden dimension của chúng. Trong computer vision, hidden dimensions được gọi là *channels*, tuy nhiên trong NLP ta sẽ gọi chúng là hidden dimensions. 

> Câu hỏi đặt ra ở đây là: tại sao chúng ta lại nhân đôi kích thước đầu ra cua convolutional filter so với đầu vào? Lý do là ở đây, chúng ta sử dụng một activation function đặc biệt có tên là *gated linear units* (GLU). GLUs tương tự như LSTM hay GRU, đều có cơ chế *gating*. GLUs sẽ trả về đầu ra có kích thước giảm một nửa => chúng ta cần nhân đôi đề đầu vào và đầu ra có kích thước bằng nhau.

> Sau khi đi qua GLUs activation function, hidden dimension size của mỗi token là bằng nhau trước và sau khi đi qua convolutional blocks. Tiếp theo, chúng được *elementwise summed* với chính vector của nó trước khi được truyền vào convolutional layer.

> Trên đây là hoạt động của single convlutional block. Các blocks tiếp theo có input là output của previous block và thực hiện công việc tương tự. Mỗi block có một bộ parameters riêng của nó, các bộ parameters này không được shared giữa các blocks. Output của last block sẽ quay trở lại encoder và được đưa vào linear layer để sinh ra *conved* vector, sau đó được *elementwise summed* với embedding của token để sinh ra *combined* vector.

##### Encoder Implementation

> Để đơn giản hóa, ta chỉ sử dụng filter có size là số lẻ - việc này cho phép padding source sentence ở cả 2 phía bằng nhau.

> Biến `scale` được sử dụng bởi tác giả để "đảm bảo phương sai của network không bị thay đổi quá nhiều". Hiệu suất của model sẽ tay đổi khá nhiều nếu sử dụng seed random khác nhau.

> Positional embedding được khởi tạo để xử lý lý "vocabulary" có kích thước bằng 100. Có nghĩa là nó có thể xử lý sequences có chiều dài tối đa 100 tokens, từ vị trí 0 tới 99. Ta có thể tằng kích thước này nếu sequence có chiều dài lớn hơn.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        assert kernel_size % 2 == 1, "Kernel size must be odd!"
        
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(input_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size, 
                                              padding = (kernel_size - 1) // 2)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        
        #src = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        #create position tensor
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [0, 1, 2, 3, ..., src len - 1]
        
        #pos = [batch size, src len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(src)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = pos_embedded = [batch size, src len, emb dim]
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, src len, emb dim]
        
        #pass embedded through linear layer to convert from emb dim to hid dim
        conv_input = self.emb2hid(embedded)
        
        #conv_input = [batch size, src len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #conv_input = [batch size, hid dim, src len]
        
        #begin convolutional blocks...
        
        for i, conv in enumerate(self.convs):
            #pass through convolutional layer
            conved = conv(self.dropout(conv_input))

            #conved = [batch size, 2 * hid dim, src len]

            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale

            #conved = [batch size, hid dim, src len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
        
        #...end convolutional blocks
        
        #permute and convert back to emb dim
        conved = self.hid2emb(conved.permute(0, 2, 1))
        
        #conved = [batch size, src len, emb dim]
        
        #elementwise sum output (conved) and input (embedded) to be used for attention
        combined = (conved + embedded) * self.scale
        
        #combined = [batch size, src len, emb dim]
        
        return conved, combined

##### Decoder

> Decoder đọc actual target sentence và cố gắng predict nó. Decoder model có sự khác biệt so với RNN models được sử dụng từ trước - lần này, decoder thực hiện predict tất cả các token trong một lần, quá trình predict các tokens được thực hiện song song với nhau. Không có sequential processing, không có decoding loop, ...

> Decoder khá giống với encoder, với một chút thay đổi về convolutional block bên trong model.

![figure4](./images/5.dec.PNG)

> Đầu tiên, embeddings không có residual connection đảm nhận việc kết nối sau convolutional blocks và transformation. Thay vào đó, embedding được truyền vào convolutional blocks để được sử dụng giống như một residual connections.

> Thứ 2, để truyền thông tin từ encoder sang decoder, encoder conved và combined output được sử dụng bên trong convolutional blocks.

> Cuối cùng, output của decoder là một linear layer chuyển từ embedding dimension sang output dimension. Nó được sử dụng để đưa ra dự đoán về từ tiếp theo trong quá trinh dịch.

##### Decoder Convolutional Blocks

> Convolutional blocks bên trong encoder cũng có một vài thay đổi.

![figure5](./images/5.dec_conv.PNG)

> Đầu tiên là padding. Thay vì padding ở 2 phía của sentence, chúng ta chỉ padding vào đầu của sentence. Do chúng ta đang sử dụng đồng thời tất cả các targets một cách song song (không phải tuần tự), nên chúng ta cần một phương pháp cho phép filters dịch token thứ i chỉ được quan sát các tokens nằm trước từ thứ i. Nếu các filters quan sát được từ phía sau, chúng sẽ sao chép các kết quả và không học được gì.